In [5]:
%pip install numpy==1.21.6

  Using cached numpy-1.21.6-cp310-cp310-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.21.6 which is incompatible.
scipy 1.15.2 requires numpy<2.5,>=1.23.5, but you have numpy 1.21.6 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.21.6 which is incompatible.
pandas 2.2.3 requires numpy>=1.22.4; python_version < "3.11", but you have numpy 1.21.6 which is incompatible.
matplotlib 3.10.1 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.
contourpy 1.3.1 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 991.0 kB/s eta 0:00:13
      --------------------------------------- 0.2/12.9 MB 2.5 MB/s eta 0:00:06
     - -------------------------------------- 0.4/12.9 MB 2.8 MB/s eta 0:00:05
     - -------------------------------------- 0.6/12.9 MB 3.6 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/12.9 MB 4.7 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.9 MB 5.3 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.9 MB 5.6 MB/s eta

  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      C:\Users\LT64\AppData\Local\Temp\pip-build-env-8lv5zxr5\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              *****************************************

In [1]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split as surprise_split
import numpy as np


def evaluate_mf_model(train_df, test_df):
    # Ánh xạ id
    user_ids = train_df["userId"].unique()
    product_ids = train_df["productId"].unique()

    user_to_index = {uid: idx for idx, uid in enumerate(user_ids)}
    product_to_index = {pid: idx for idx, pid in enumerate(product_ids)}

    # Chuẩn hóa dataframe
    train_df["user_index"] = train_df["userId"].map(user_to_index)
    train_df["product_index"] = train_df["productId"].map(product_to_index)

    test_df["user_index"] = test_df["userId"].map(user_to_index)
    test_df["product_index"] = test_df["productId"].map(product_to_index)

    # Tạo Surprise Dataset
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(
        train_df[["user_index", "product_index", "rating"]], reader
    )
    trainset = data.build_full_trainset()

    # Tạo mô hình MF (SVD)
    algo = SVD(n_factors=50, lr_all=0.01, reg_all=0.1)
    algo.fit(trainset)

    # Đánh giá trên test set
    testset = list(
        zip(test_df["user_index"], test_df["product_index"], test_df["rating"])
    )
    predictions = [algo.predict(uid, iid, r_ui) for (uid, iid, r_ui) in testset]
    rmse = accuracy.rmse(predictions, verbose=True)

    # Trích xuất ma trận đặc trưng P và Q
    P = algo.pu  # ma trận user factors
    Q = algo.qi  # ma trận item factors

    return P, Q, user_to_index, product_to_index

ModuleNotFoundError: No module named 'surprise'